In [43]:
!pip install symspellpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 5.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 kB 2.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for editdistpy: filename=editdistpy-0.1.3-cp310-cp310-linux_x86_64.whl size=130264 sha256=3d4b39b643963edeaebe05cccdcc0856ee7bbe909640345ce543feea7cf0a06b
  Stored in directory: /home/nikita/.cache/pip/wheels/88/6a/a6/a1283cc145323a1fb3d475bd158ee60b248ab1985230d266fc
Successfully built editdistpy

[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


In [95]:
import pydub
import librosa
import nemo.collections.asr as nemo_asr
import glob
import nemo.collections.tts as nemo_tts
import numpy as np
import torch
import pyctcdecode
from word_beam_search import WordBeamSearch
import itertools
import symspellpy
import matplotlib.pyplot as plt
from forced_alignment import get_trellis, backtrack, plot_trellis_with_path, merge_repeats, plot_trellis_with_segments, merge_words
import pickle
from IPython.display import Audio

In [116]:
models = [
    'stt_en_quartznet15x5',
    #'QuartzNet15x5Base-En'
    #'stt_en_jasper10x5dr'
]
#for model in models:
asr = nemo_asr.models.EncDecCTCModel.from_pretrained(models[0], strict=False).eval().cpu()
classifier = nemo_asr.models.EncDecClassificationModel.from_pretrained("commandrecognition_en_matchboxnet3x2x64_v2").to('cpu').eval()
config =asr.to_config_dict()
labels = config['decoder']['vocabulary'] + ['']
svm = pickle.load(open("svm.pkl", 'rb'))

[NeMo I 2022-12-16 20:16:57 cloud:56] Found existing object /home/nikita/.cache/torch/NeMo/NeMo_1.13.0/stt_en_quartznet15x5/16661021d16e679bdfd97a2a03944c49/stt_en_quartznet15x5.nemo.
[NeMo I 2022-12-16 20:16:57 cloud:62] Re-using file from: /home/nikita/.cache/torch/NeMo/NeMo_1.13.0/stt_en_quartznet15x5/16661021d16e679bdfd97a2a03944c49/stt_en_quartznet15x5.nemo
[NeMo I 2022-12-16 20:16:57 common:911] Instantiating model from pre-trained checkpoint


[NeMo W 2022-12-16 20:16:58 modelPT:142] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data2/voices/train_1k.json
    sample_rate: 16000
    labels:
    - ' '
    - a
    - b
    - c
    - d
    - e
    - f
    - g
    - h
    - i
    - j
    - k
    - l
    - m
    - 'n'
    - o
    - p
    - q
    - r
    - s
    - t
    - u
    - v
    - w
    - x
    - 'y'
    - z
    - ''''
    batch_size: 32
    trim_silence: true
    max_duration: 16.7
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: /asr_set_1.2/train/train_{0..1023}.tar
    num_workers: 20
    
[NeMo W 2022-12-16 20:16:58 modelPT:149] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
   

[NeMo I 2022-12-16 20:16:58 features:225] PADDING: 16
[NeMo I 2022-12-16 20:16:58 save_restore_connector:243] Model EncDecCTCModel was successfully restored from /home/nikita/.cache/torch/NeMo/NeMo_1.13.0/stt_en_quartznet15x5/16661021d16e679bdfd97a2a03944c49/stt_en_quartznet15x5.nemo.
[NeMo I 2022-12-16 20:16:58 cloud:56] Found existing object /home/nikita/.cache/torch/NeMo/NeMo_1.13.0/commandrecognition_en_matchboxnet3x2x64_v2/8d5735c9c20648d313720dd95300978d/commandrecognition_en_matchboxnet3x2x64_v2.nemo.
[NeMo I 2022-12-16 20:16:58 cloud:62] Re-using file from: /home/nikita/.cache/torch/NeMo/NeMo_1.13.0/commandrecognition_en_matchboxnet3x2x64_v2/8d5735c9c20648d313720dd95300978d/commandrecognition_en_matchboxnet3x2x64_v2.nemo
[NeMo I 2022-12-16 20:16:58 common:911] Instantiating model from pre-trained checkpoint


[NeMo W 2022-12-16 20:16:58 modelPT:142] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: null
    sample_rate: 16000
    labels:
    - visual
    - wow
    - learn
    - backward
    - dog
    - two
    - left
    - happy
    - nine
    - go
    - up
    - bed
    - stop
    - one
    - zero
    - tree
    - seven
    - 'on'
    - four
    - bird
    - right
    - eight
    - 'no'
    - six
    - forward
    - house
    - marvin
    - sheila
    - five
    - 'off'
    - three
    - down
    - cat
    - follow
    - 'yes'
    batch_size: 128
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      shift:
        prob: 1.0
        min_shift_ms: -5.0
        max_shift_ms: 5.0
      white_noise:
        prob: 1.0
        min_level: -90
        max_level: -46


[NeMo I 2022-12-16 20:16:58 save_restore_connector:243] Model EncDecClassificationModel was successfully restored from /home/nikita/.cache/torch/NeMo/NeMo_1.13.0/commandrecognition_en_matchboxnet3x2x64_v2/8d5735c9c20648d313720dd95300978d/commandrecognition_en_matchboxnet3x2x64_v2.nemo.


In [125]:
bag = set()
with open("train_text.txt") as f:
    t=f.readlines()
    for line in t:
        for word in line[:-1].split():
            bag.add(word)
bag.add('all')
bag.add('on')
bag.add('lingual')


decoder = pyctcdecode.build_ctcdecoder(labels, "text.arpa", alpha=0.5, beta=1.5, lm_score_boundary=True, unigrams=list(bag))
sm = symspellpy.SymSpell(max_dictionary_edit_distance=2)
sm.create_dictionary(corpus="train_text.txt")

Only 36 unigrams passed as vocabulary. Is this small or artificial data?
Loading the LM will be faster if you build a binary file.
Reading /home/nikita/Documents/denti/text.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************


True

In [126]:
for f in glob.glob("test/*.wav"):
    seg = pydub.AudioSegment.from_file(f)
    samples = seg.get_array_of_samples()
    samples = np.array(samples).astype(np.float32)
    lengths = torch.IntTensor([len(samples)]).to('cpu')
    samples = torch.FloatTensor(samples.reshape(1,-1)).to('cpu')
    with torch.no_grad():
        logs,_,tops = asr.forward(input_signal=samples, input_signal_length=lengths)
    transcript = [labels[int(k)] for (k, g) in itertools.groupby(tops[0].numpy())]
    baseline_text = ''.join(transcript)
    print(baseline_text)
    decoded_text = decoder.decode(logs[0].numpy(), hotwords=['mesial', 'light', 'margin', 'gingival', 'lingual'], hotword_weight=20)
    print(decoded_text)
    final_text = []
    for word in decoded_text.split():
        if word not in bag:
            correct = sm.lookup_compound(word, 2)[0].term if len(sm.lookup(word.lower(), 2)) else word
            final_text.append(correct)
        else:
            final_text.append(word)
    final_text = " ".join(final_text)

    #final_text = [sm.lookup_compound(w, 2)[0].term if len(sm.lookup(w.lower(), 2)) else w for w in decoded_text.split()]
    print(final_text)

    labels = config['decoder']['vocabulary'] + ['']
    labels[0] = '|'
    transcript = [labels[int(k)] for (k, g) in itertools.groupby(tops[0].numpy())]
    dictionary = {c: i for i, c in enumerate(labels)}
    dictionary[''] = 0
    dictionary['|'] = 28
    tokens = [dictionary[c] for c in transcript]
    trellis = get_trellis(logs[0], tokens)
    path = backtrack(trellis, logs[0], tokens)
    segments = merge_repeats(path, transcript)
    word_segments = merge_words(segments)
    y,_ = librosa.load(f, sr=16_000)
    l = len(y)
    for word in word_segments:
        start = word.start/logs.shape[1]
        end = word.end/logs.shape[1]
        st = int(start*l)
        end =int(end*l)
        y0 = y[st:end]
        input_signal = torch.FloatTensor([y0])
        input_signal_length = torch.IntTensor([len(y0)])
        res = classifier.forward(input_signal=input_signal, input_signal_length=input_signal_length)
        X_vector = res.detach().numpy()[0]
        class_ = svm.predict(X_vector.reshape(1,-1))
        print(class_)

vocation|eighteen|grade|one
furcation eighteen grade one
furcation eighteen grade one
[2]
[2]
[3]
[3]
jump|gingerval|mandin
jump gingerval margin
jump gingival margin
[3]
[5]
[3]
clack|light|on|all
plaque light on all
plaque light on all
[5]
[5]
[3]
[5]
bleeding|thirteen|meseolingual
bleeding thirteen mesialingual
bleeding thirteen mesialingual
[3]
[3]
[5]
